In [1]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="LST20_Corpus")

Using custom data configuration default-data_dir=LST20_Corpus
Reusing dataset lst20 (C:\Users\pitiw\.cache\huggingface\datasets\lst20\default-data_dir=LST20_Corpus\0.0.0\e1b2a921fb011578ab43ddbbf789f3c500d62cb2df8ae4ed4b60bae8e4c0d3ad)
100%|██████████| 3/3 [00:00<00:00, 25.86it/s]


In [2]:
lst20

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [3]:
import pandas as pd
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [17]:
# See tags from https://github.com/huggingface/datasets/blob/master/datasets/lst20/lst20.py
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]
_NER_TAGS = [
        "O",
        "B_BRN",
        "B_DES",
        "B_DTM",
        "B_LOC",
        "B_MEA",
        "B_NUM",
        "B_ORG",
        "B_PER",
        "B_TRM",
        "B_TTL",
        "I_BRN",
        "I_DES",
        "I_DTM",
        "I_LOC",
        "I_MEA",
        "I_NUM",
        "I_ORG",
        "I_PER",
        "I_TRM",
        "I_TTL",
        "E_BRN",
        "E_DES",
        "E_DTM",
        "E_LOC",
        "E_MEA",
        "E_NUM",
        "E_ORG",
        "E_PER",
        "E_TRM",
        "E_TTL",
    ]
_CLAUSE_TAGS = ["O", "B_CLS", "I_CLS", "E_CLS"]

In [18]:
list(map(lambda x: len(x), train_df["tokens"]))

[12,
 9,
 26,
 20,
 8,
 29,
 39,
 41,
 44,
 53,
 82,
 39,
 12,
 9,
 15,
 30,
 46,
 35,
 63,
 22,
 32,
 32,
 21,
 9,
 29,
 30,
 17,
 47,
 27,
 98,
 26,
 19,
 4,
 4,
 4,
 4,
 6,
 35,
 67,
 43,
 19,
 9,
 36,
 10,
 9,
 13,
 36,
 12,
 67,
 10,
 9,
 32,
 66,
 65,
 24,
 37,
 58,
 78,
 43,
 27,
 52,
 23,
 34,
 44,
 50,
 30,
 56,
 14,
 59,
 51,
 45,
 13,
 9,
 18,
 46,
 25,
 26,
 13,
 50,
 13,
 29,
 29,
 19,
 37,
 26,
 29,
 26,
 16,
 13,
 9,
 23,
 8,
 27,
 23,
 34,
 40,
 29,
 29,
 41,
 57,
 34,
 29,
 38,
 17,
 20,
 43,
 18,
 15,
 19,
 17,
 16,
 17,
 14,
 19,
 16,
 9,
 34,
 15,
 47,
 27,
 23,
 43,
 20,
 18,
 41,
 30,
 28,
 33,
 22,
 42,
 10,
 29,
 12,
 22,
 18,
 20,
 25,
 29,
 12,
 38,
 28,
 27,
 11,
 28,
 27,
 19,
 23,
 24,
 13,
 11,
 9,
 15,
 45,
 35,
 30,
 23,
 46,
 16,
 28,
 42,
 23,
 41,
 31,
 15,
 50,
 50,
 33,
 28,
 15,
 24,
 9,
 35,
 35,
 33,
 32,
 34,
 40,
 36,
 18,
 40,
 22,
 48,
 49,
 37,
 18,
 35,
 35,
 53,
 60,
 44,
 61,
 35,
 35,
 63,
 6,
 78,
 18,
 31,
 14,
 46,
 9,
 18,
 14,
 107,

In [19]:
train_df["ner_tags"][0]

[8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]

In [20]:
list(map(lambda x: _NER_TAGS[x], train_df["ner_tags"][0]))

['B_PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_ORG', 'O', 'O', 'O']

In [21]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  #Limit at 1000 rows for speed.
  for (idx, r) in df[:500].iterrows():
    # print(idx)
    for (i, t) in enumerate(r['tokens']):
      # print(i,t)
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])

    return pd.DataFrame(
        {"sentence_id": sentence_id, "words": words, "labels": labels}
    )


In [24]:
train_df

,id,fname,tokens,pos_tags,ner_tags,clause_tags
0,0,T00126.txt,"[สุรยุทธ์, ยัน, ปฏิเสธ, ลงนาม, _, MOU, _, กับ,...","[0, 1, 1, 1, 2, 0, 2, 4, 0, 12, 1, 0]","[8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]"
1,1,T00126.txt,"[1, _, กันยายน, _, 2550, _, 12:21, _, น.]","[8, 2, 0, 2, 8, 2, 8, 2, 10]","[3, 13, 13, 13, 23, 0, 3, 13, 23]","[1, 2, 2, 2, 2, 2, 2, 2, 3]"
2,2,T00126.txt,"[นายก, ฯ, _, ย้ำ, ไม่, ลงนาม, ใน, _, MOU, _, ร...","[0, 2, 2, 1, 12, 1, 4, 2, 0, 2, 4, 0, 3, 5, 6,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, ..."
3,3,T00126.txt,"[ยัน, ไทย, ไม่, ได้, ล้มเหลว, ถึง, ขั้น, นั้น,...","[1, 0, 12, 5, 1, 1, 0, 9, 1, 7, 1, 0, 9, 12, 1...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,4,T00126.txt,"[พร้อม, หนุน, กกต., จัด, เลือกตั้ง, ให้, บริสุ...","[3, 1, 0, 1, 1, 3, 1, 1]","[0, 0, 7, 0, 0, 0, 0, 0]","[1, 2, 2, 2, 3, 1, 2, 3]"
...,...,...,...,...,...,...
63305,6,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, หาก, การ, ยื่น...","[0, 0, 2, 1, 3, 2, 3, 7, 1, 0, 12, 1, 3, 5, 1,...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 3, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63306,7,T13136.txt,"[เมื่อ, ถาม, ว่า, _, ใน, ทาง, การเมือง, _, หาก...","[4, 1, 3, 2, 4, 0, 0, 2, 3, 0, 3, 1, 0, 6, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63307,8,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, เรื่อง, กฎหมาย...","[0, 0, 2, 1, 3, 2, 0, 0, 5, 3, 1, 2, 7, 1, 1, ...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63308,9,T13136.txt,"[หาก, เป็น, การ, ขอ, ถวาย, ฎีกา, ตาม, โบราณ, ร...","[3, 1, 7, 1, 1, 0, 4, 0, 0, 2, 5, 1, 0, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, ..."


In [22]:
train_ = convert_to_simple_transformer_format(train_df, "ner_tags", _NER_TAGS)

In [23]:
train_

,sentence_id,words,labels
0,0,สุรยุทธ์,B_PER
1,0,ยัน,O
2,0,ปฏิเสธ,O
3,0,ลงนาม,O
4,0,_,O
5,0,MOU,O
6,0,_,O
7,0,กับ,O
8,0,อียู,B_ORG
9,0,ไม่,O


In [49]:
validation_ = convert_to_simple_transformer_format(validation_df, "ner_tags", _NER_TAGS)

In [50]:
test_ = convert_to_simple_transformer_format(test_df, "ner_tags", _NER_TAGS)

In [13]:
import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
ner_args = NERArgs()
ner_args.train_batch_size = 12
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.num_train_epochs = 1 #10

model = NERModel(
    "bert", "monsoon-nlp/bert-base-thai", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the model
model.train_model(train_, eval_data=validation_)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized

(84,
 defaultdict(list,
             {'global_step': [84],
              'train_loss': [0.8981155753135681],
              'eval_loss': [1.3495024011135102],
              'precision': [0.1371794871794872],
              'recall': [0.035900016775708775],
              'f1_score': [0.05690732615343704]}))

In [14]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["ประยุกธ์ เผย ลูกน้อง ไม่ เชื่อ พูด ตลก ได้ สามารถ แข่ง กับ โน๊ต อุดม เดี่ยว ไมโครโฟน เมื่อ วันที่ 26 กุมภาพันธ์ ที่ ตลาด หลักทรัพย์ แห่ง ประเทศไทย"])
predictions

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 27.77it/s]


[[{'ประยุกธ์': 'O'},
  {'เผย': 'O'},
  {'ลูกน้อง': 'O'},
  {'ไม่': 'O'},
  {'เชื่อ': 'O'},
  {'พูด': 'O'},
  {'ตลก': 'O'},
  {'ได้': 'O'},
  {'สามารถ': 'O'},
  {'แข่ง': 'O'},
  {'กับ': 'O'},
  {'โน๊ต': 'O'},
  {'อุดม': 'O'},
  {'เดี่ยว': 'O'},
  {'ไมโครโฟน': 'O'},
  {'เมื่อ': 'O'},
  {'วันที่': 'O'},
  {'26': 'O'},
  {'กุมภาพันธ์': 'O'},
  {'ที่': 'O'},
  {'ตลาด': 'O'},
  {'หลักทรัพย์': 'O'},
  {'แห่ง': 'O'},
  {'ประเทศไทย': 'O'}]]

In [15]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_)
result

Running Evaluation: 100%|██████████| 125/125 [00:16<00:00,  7.40it/s]
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: E_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_DTM

{'eval_loss': 0.8016529335975647,
 'precision': 0.29343629343629346,
 'recall': 0.05874774542643649,
 'f1_score': 0.09789609274366681}

In [33]:
test_ner = NERModel("bert", './outputs', args=ner_args)

In [34]:
# Make predictions with the loaded model
predictions, raw_outputs = test_ner.predict(["ประยุกธ์ เผย ลูกน้อง ไม่ เชื่อ พูด ตลก ได้ สามารถ แข่ง กับ โน๊ต อุดม เดี่ยว ไมโครโฟน เมื่อ วันที่ 26 กุมภาพันธ์ ที่ ตลาด หลักทรัพย์ แห่ง ประเทศไทย"])
predictions

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 31.10it/s]


[[{'ประยุกธ์': 'O'},
  {'เผย': 'O'},
  {'ลูกน้อง': 'O'},
  {'ไม่': 'O'},
  {'เชื่อ': 'O'},
  {'พูด': 'O'},
  {'ตลก': 'O'},
  {'ได้': 'O'},
  {'สามารถ': 'O'},
  {'แข่ง': 'O'},
  {'กับ': 'O'},
  {'โน๊ต': 'O'},
  {'อุดม': 'O'},
  {'เดี่ยว': 'O'},
  {'ไมโครโฟน': 'O'},
  {'เมื่อ': 'O'},
  {'วันที่': 'O'},
  {'26': 'O'},
  {'กุมภาพันธ์': 'O'},
  {'ที่': 'O'},
  {'ตลาด': 'O'},
  {'หลักทรัพย์': 'O'},
  {'แห่ง': 'O'},
  {'ประเทศไทย': 'O'}]]